## 10 - Computação Bayesiana Aproximada

### 10.1 - Hipóteses de variabilidade

- Variabilidade masculina é maior;

- Conclusão tendênciosa, e evidências da hipótese de variabilidade são fracas;

- O conjunto de dados do Sistema de Vigilância de Fatores Comportamentais, inclui respostas de 154407 homens e 254722 mulheres:

    - Para as mulheres:
        - A altura média é 163 cm;
        - O desvio padrão é 7,3 cm;
        - O coeficiente de variação é 0,0444;


    - Para os homens:
        - A altura média é 178 cm; 
        - O desvio padrão é 7,7 cm;
        - O coeficiente de variação é 0,0433;

- Vou seguir algumas etapas:
    - Começaremos com a implementação mais simples;
    - Ao calcular probabilidades sob uma transformação de log, podemos escalar até o tamanho total do conjunto de dados, mas a computação fica lenta.
    - **Computação Bayesiana Aproximada** (ABC).

### 10.2 - Média e desvio padrão

- Distribuição conjunta para estimar os parâmetros de uma distribuição gaussiana: a média, mu e o desvio padrão, sigma;

In [7]:
import thinkbayes
class Height(thinkbayes.Suite, thinkbayes.Joint):

    def __init__(self, mus, sigmas):
        pairs = [(mu, sigma) 
                 for mu in mus
                 for sigma in sigmas]

        thinkbayes.Suite.__init__(self, pairs)
        
    def Likelihood(self, data, hypo):
        x = data
        mu, sigma = hypo
        like = thinkbayes.EvalGaussianPdf(x, mu, sigma)
        return like

- A distribuição a priori é uniforme para todos os pares (mu, sigma).
- Dados os valores hipotéticos de mu e sigma, calculamos a verossimilhança de um valor específico, x.

- faixas apropriadas para mus e sigmas?

In [8]:
def FindPriorRanges(xs, num_points, num_stderrs=3.0):

    # compute m and s
    n = len(xs)
    m = numpy.mean(xs)
    s = numpy.std(xs)

    # compute ranges for m and s
    stderr_m = s / math.sqrt(n)
    mus = MakeRange(m, stderr_m, num_stderrs)

    stderr_s = s / math.sqrt(2 * (n-1))
    sigmas = MakeRange(s, stderr_s, num_stderrs)

    return mus, sigmas

- **xs** é o conjunto de dados;
- **num_points** é o número desejado de valores no intervalo;
- **num_stderrs** é a largura do intervalo em cada lado da estimativa, em número de erros padrão.

In [10]:
def MakeRange(estimate, stderr, num_stderrs):
    spread = stderr * num_stderrs
    array = numpy.linspace(estimate-spread, estimate+spread, num_points)
    return array

### 10.3 - Update

In [13]:
def update(xs, num_points):
    mus, sigmas = FindPriorRanges(xs, num_points)
    suite = Height(mus, sigmas)
    suite.UpdateSet(xs)
    print(suite.MaximumLikelihood())

### 10.4 - A distribuição posteriori de CV

- A partir da distribuição posteriori de mu e sigma;
- Calculamos a distribuição de CV para homens e mulheres; 
- A probabilidade de que um exceda o outro.

In [16]:
def CoefVariation(suite):
    pmf = thinkbayes.Pmf()
    for (mu, sigma), p in suite.Items():
        pmf.Incr(sigma/mu, p)
    return pmf

- Algums problemas:
    - Problemas computacionais devido às limitações sobre ponto flutuante;
    - Presença de valores extremos;

### 10.5 - Subfluxo

- **subfluxo**: Valor muito pequeno que não pode ser representado por um número de ponto flutuante; portanto, é arredondado para zero;

- Algumas soluções:
    - Renormalizar após cada atualização ou após cada lote de 100;
    - Calcular as verossimilhança sob uma transformação de log;

In [19]:
# class Pmf

    def Log(self):
        m = self.MaxLike()
        for x, p in self.d.iteritems():
            if p:
                self.Set(x, math.log(p/m))
            else:
                self.Remove(x)

    def Exp(self):
        m = self.MaxLike()
        for x, p in self.d.iteritems():
            self.Set(x, math.exp(p-m))

### 10.6 Log Verossimilhança

In [21]:
def LogLikelihood(self, data, hypo):
    x = data
    mu, sigma = hypo
    loglike = scipy.stats.norm.logpdf(x, mu, sigma)
    return loglike

In [22]:
def update(xs):
    suite.Log()
    suite.LogUpdateSet(xs)
    suite.Exp()
    suite.Normalize()

### 10.7 - Um pouco de otimização

### 10.8 - ABC

- A motivação por trás do ABC é que a probabilidade de qualquer conjunto de dados específico é:
    - Muito pequeno, especialmente para conjuntos de dados grandes, por isso tivemos que usar a transformação de log;
    - Caro para calcular, é por isso que tivemos que fazer tanta otimização;
    - Não é realmente o que queremos de qualquer maneira.

É mais relevante perguntar: "Se amostrarmos 100 mil pessoas de uma população com valores hipotéticos de µ e σ, qual seria a chance de coletar uma amostra com a média e variância observadas?"

- Para uma distribuição Gaussiana (µ, σ)
    - A distribuição da média é uma gaussiana com os parâmetros µ e $\frac{σ}{\sqrt n}$.
    - a distribuição do desvio padrão é uma gaussiana com os parâmetros σ e $\frac{σ}{\sqrt(2(n −1))}$.

In [23]:
def LogUpdateSetABC(self, data):
    xs = data
    n = len(xs)

    # compute sample statistics
    m = numpy.mean(xs)
    s = numpy.std(xs)

    for hypo in sorted(self.Values()):
        mu, sigma = hypo

        # compute log likelihood of m, given hypo
        stderr_m = sigma / math.sqrt(n)
        loglike = EvalGaussianLogPdf(m, mu, stderr_m)

        #compute log likelihood of s, given hypo
        stderr_s = sigma / math.sqrt(2 * (n-1))
        loglike += EvalGaussianLogPdf(s, sigma, stderr_s)

        self.Incr(hypo, loglike)

### 10.9 - Estimatição robusta

- Há vários outliers neste conjunto de dados que quase certamente são erros;
- Não é impossível que esses valores estejam corretos, mas é improvável;

- Poderíamos usar a mediana e a faixa inter-quartil (IQR).

In [24]:
def MedianIPR(xs, p):
    cdf = thinkbayes.MakeCdfFromList(xs)
    median = cdf.Percentile(50)

    alpha = (1-p) / 2
    ipr = cdf.Value(1-alpha) - cdf.Value(alpha)
    return median, ipr

- Converter de **ipr** para uma estimativa de sigma

In [29]:
def MedianS(xs, num_sigmas):
    half_p = thinkbayes.StandardGaussianCdf(num_sigmas) - 0.5

    median, ipr = MedianIPR(xs, half_p * 2)
    s = ipr / 2 / num_sigmas

    return median, s

![](dist_contorno_man.png)![](dist_contorno_woman.png)

### 10.10 - Quem é mais variável?

![](dist_var_man_woman.png)
- Distribuição posterior do CV
- O coeficiente de variação é maior para homens do que para mulheres?

- A média para os homens é 0,0410; 
- A média para as mulheres é 0,0429.

- Com num_sigmas=1, concluímos que as mulheres são mais variáveis;
- Com num_sigmas=2, concluímos que os homens são mais variáveis;

- Avaliação da hipótese de variabilidade depende da interpretação da “variabilidade”:
    - num_sigmas=1, focamos nas pessoas próximas à média;
    - Com o aumento de num_sigmas, damos mais peso aos extremos.